In [1]:
import pandas as pd
f1='/Users/markuswiikjensen/Desktop/Master-Data/Operating data/2020/pono2001.sdv'

df = pd.read_csv(f1, delimiter='\t', encoding='ISO-8859-1' )
df

,"# Uke nr. 1 - 2020 : Priser, omsetning og forbruk i Norge"
0,#
1,# Datatype(ST);Aar;Uke;Dag;Time;Tot.ant.timer;...
2,# Datatype(BE);Alias;Tekst
3,# Datatype;Kode;Aar;Uke;Dag;Dato;Alias;Time1;T...
4,# Datatype(AL);Antall linjer paa fila.
...,...
840,UT;U;2020;1;7;05.01.2020;NO_RU;25;25;26;;26;26...
841,UT;U;2020;1;7;05.01.2020;NO2_NL;-375;-360;-10;...
842,UT;U;2020;1;7;05.01.2020;NO_NL;-375;-360;-10;;...
843,UT;U;2020;1;7;05.01.2020;NO;-195;107;516;;812;...


In [3]:
def clean_operating_data(df, kode):
    df=df.iloc[:,0].str.split(';', expand=True)
    df.columns=df.iloc[3]
    df=df.iloc[47:-1]
    keep_list = ['NO1', 'NO2', 'NO3', 'NO4', 'NO5', 'NO6']

    df = df[df['Alias'].str.contains('|'.join(keep_list))]

    date_col = 'Dato'
    hour_cols = ['Time1',
           'Time2', 'Time3', 'Time4', 'Time5', 'Time6', 'Time7',
           'Time8', 'Time9', 'Time10', 'Time11', 'Time12', 'Time13', 'Time14',
           'Time15', 'Time16', 'Time17', 'Time18', 'Time19', 'Time20', 'Time21',
           'Time22', 'Time23', 'Time24']
    df.drop('Time3B', axis=1, inplace=True)
    df['Time3']=df['Time3A']

    # unpivot the hourly observation columns into two new columns: hour and value
    df_melted = pd.melt(df, id_vars=[date_col, 'Kode', 'Alias'], value_vars=hour_cols, var_name='hour', value_name='value')

    # replace hour values of 24 with 00
    df_melted['hour'] = df_melted['hour'].str.strip('Time').str.replace('^24', '00', regex=True)

    # combine the date and hour columns into a single datetime column
    df_melted['datetime'] = pd.to_datetime(df_melted[date_col] + ' ' + df_melted['hour'].str.strip('Time') + ':00:00', format='%d.%m.%Y %H:%M:%S')

    # drop the original date and hour columns, as well as any other unnecessary columns
    df_melted.drop(columns=[date_col, 'hour'], inplace=True)

    # sort the data by the datetime column
    df_melted.sort_values('datetime', inplace=True)

    pr = df_melted['Kode']==kode
    df_pr = df_melted[pr]

    # create a pivot table with Alias as columns and datetime as index, including all columns
    pivot_table = df_pr.pivot_table('value', 'datetime','Alias')
    
    return pivot_table



In [4]:
production_df= clean_operating_data(df, 'P')
production_df

Alias,NO1,NO2,NO3,NO4,NO5
datetime,,,,,
2019-12-30 00:00:00,1882.0,3554.0,2058.0,1604.0,2388.0
2019-12-30 01:00:00,1775.0,3540.0,1929.0,1414.0,2574.0
2019-12-30 02:00:00,1759.0,3411.0,1925.0,1480.0,2544.0
2019-12-30 03:00:00,1739.0,3491.0,1903.0,1429.0,2423.0
2019-12-30 04:00:00,1745.0,3490.0,1915.0,1423.0,2355.0
...,...,...,...,...,...
2020-01-05 19:00:00,2329.0,5769.0,3184.0,3582.0,4318.0
2020-01-05 20:00:00,2247.0,5536.0,3172.0,3555.0,3944.0
2020-01-05 21:00:00,2075.0,5007.0,3090.0,3513.0,3430.0


In [5]:
def combine_clean_data(list_of_tables, kode):
    
    # Initialize an empty list to store the cleaned tables
    list_of_clean_tables = []
    
    for i in list_of_tables:
        new_i = clean_operating_data(i, kode)
        # Append the cleaned table to the list of cleaned tables
        list_of_clean_tables.append(new_i)
        
    # Append the tables together
    combined_table = pd.concat(list_of_clean_tables, axis=0, ignore_index=False)
    
    return combined_table


In [11]:

import os

folder_path = '/Users/markuswiikjensen/Desktop/Master-Data/Operating data/2020'  # replace with the path to your folder
dfs = []  # initialize an empty list to store the DataFrames

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path, delimiter='\t', encoding='ISO-8859-1' )  # read the CSV file as a DataFrame
    dfs.append(df)  # append the DataFrame to the list

# print the list of DataFrames



In [12]:
pr1=combine_clean_data(dfs, 'P')

/var/folders/qt/z8wfyvqx1l55qqc_y_5n83fc0000gn/T/ipykernel_14460/2034803777.py:37: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  pivot_table = df_pr.pivot_table('value', 'datetime','Alias')


In [14]:

folder_path = '/Users/markuswiikjensen/Desktop/Master-Data/Operating data/2021'  # replace with the path to your folder
dfs = []  # initialize an empty list to store the DataFrames

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path, delimiter='\t', encoding='ISO-8859-1' )  # read the CSV file as a DataFrame
    dfs.append(df)  # append the DataFrame to the list

# print the list of DataFrames

In [15]:
pr2=combine_clean_data(dfs, 'P')

/var/folders/qt/z8wfyvqx1l55qqc_y_5n83fc0000gn/T/ipykernel_14460/2034803777.py:37: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  pivot_table = df_pr.pivot_table('value', 'datetime','Alias')


In [17]:
folder_path = '/Users/markuswiikjensen/Desktop/Master-Data/Operating data/2022'  # replace with the path to your folder
dfs = []  # initialize an empty list to store the DataFrames

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path, delimiter='\t', encoding='ISO-8859-1' )  # read the CSV file as a DataFrame
    dfs.append(df)  # append the DataFrame to the list

# print the list of DataFrames

In [18]:
pr3=combine_clean_data(dfs, 'P')

/var/folders/qt/z8wfyvqx1l55qqc_y_5n83fc0000gn/T/ipykernel_14460/2034803777.py:37: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  pivot_table = df_pr.pivot_table('value', 'datetime','Alias')


In [20]:
folder_path = '/Users/markuswiikjensen/Desktop/Master-Data/Operating data/2023'  # replace with the path to your folder
dfs = []  # initialize an empty list to store the DataFrames

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path, delimiter='\t', encoding='ISO-8859-1' )  # read the CSV file as a DataFrame
    dfs.append(df)  # append the DataFrame to the list

# print the list of DataFrames

In [21]:
pr4=combine_clean_data(dfs, 'P')

/var/folders/qt/z8wfyvqx1l55qqc_y_5n83fc0000gn/T/ipykernel_14460/2034803777.py:37: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  pivot_table = df_pr.pivot_table('value', 'datetime','Alias')
/var/folders/qt/z8wfyvqx1l55qqc_y_5n83fc0000gn/T/ipykernel_14460/2034803777.py:37: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  pivot_table = df_pr.pivot_table('value', 'datetime','Alias')
/var/folders/qt/z8wfyvqx1l55qqc_y_5n83fc0000gn/T/ipykernel_14460/2034803777.py:37: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the fu

In [25]:
production_df=pd.concat([pr1,pr2,pr3,pr4], axis=0)
production_df.sort_index(ascending=True, inplace=True)

Alias,NO1,NO2,NO3,NO4,NO5
datetime,,,,,
2019-12-30 00:00:00,1882.0,3554.0,2058.0,1604.0,2388.0
2019-12-30 01:00:00,1775.0,3540.0,1929.0,1414.0,2574.0
2019-12-30 02:00:00,1759.0,3411.0,1925.0,1480.0,2544.0
2019-12-30 03:00:00,1739.0,3491.0,1903.0,1429.0,2423.0
2019-12-30 04:00:00,1745.0,3490.0,1915.0,1423.0,2355.0
...,...,...,...,...,...
2023-04-07 19:00:00,NaN,NaN,NaN,NaN,NaN
2023-04-07 20:00:00,NaN,NaN,NaN,NaN,NaN
2023-04-07 21:00:00,NaN,NaN,NaN,NaN,NaN


In [26]:
production_df.to_csv('electricity_production.csv')